In [1]:
import yaml
import os
import pickle

import pandas as pd
import numpy as np
import nltk
import re

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [12]:
from sklearn.metrics import f1_score

In [2]:
def preprocess_text(text):
    """
    Очистка и лемматизация текста
    Присутствует специфика данных
    """
    tokens = mystem.lemmatize(text.lower())
    
    text = [re.sub('[0-9][а-я]|\d+', '*', token) for token in tokens if (token.strip() not in russian_stopwords) & \
            (token.strip() not in eng_stopwords) & \
            (token.strip() not in punctuation) & \
            (token.strip() not in numbers) & \
            (token.strip() not in custom_top_words) & \
            # (token.strip() not in terms_to_delete) & \
            (token != ' ')]
    if '*' in text:
        text.remove('*')
        
    text = ' '.join(text)
    
    text = text.replace('*', '')
    text = text.replace('®', '')
    text = text.replace('№', '')
    text = text.replace(')', '')
    text = text.replace('(', '')
    text = text.replace('.', '')
    text = text.replace('"', '')
    text = text.replace('\n', '')
    text = text.replace('«', '')
    text = text.replace('»', '')
    text = text.replace(',,', '')
    text = text.replace(',', '')
    text = text.replace('окпд', '')
    text = text.replace('ooo', '')  # eng
    text = text.replace('ооо', '')  # rus
    text = text.split(' ')
    text = ' '.join([x for x in text if x not in ['']])
    
    # try:
    #     regex_str = [r"рег. №", "рег №", 'серия', "ру №", "лср-", "лп-", "P N".lower()]
    #     res = re.search("|".join(regex_str), text).start()
    #     text = text[:res]
    # except:
    #     pass
    
    return text

# 3 test

In [22]:
data = pd.read_excel('data/raw/3test.xlsx', engine='openpyxl')
data.head()

,id,Общее наименование продукции,Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции),Подкатегория продукции
0,140000,смеси бетонные,2364.1,Смеси строительные
1,140001,ветеринарный препарат,9300.2,Препараты и средства химико-фармацевтические д...
2,140002,"Вибуркол, суппозитории ректальные гомеопатичес...",9300.1,"Лекарственные средства, зарегистрированные в у..."
3,140003,Неозим Prot-A 200 (Neozim Prot-А 200) – кормов...,9750.1,Комбикорма и премиксы
4,140004,"Вибуркол, суппозитории ректальные гомеопатичес...",9300.1,"Лекарственные средства, зарегистрированные в у..."


In [4]:
codes = pd.read_csv('data/code-info.csv', index_col='code')
code_dict = dict()
for el in codes.iterrows():
    code_dict[el[0]] = el[1][0]

In [5]:
with open(os.getcwd() + '/app-config.yaml') as f:
    config = yaml.safe_load(f)
    
nltk.download('stopwords')
mystem = Mystem()  # инициализируем для последующего исползования

russian_stopwords = stopwords.words("russian")
eng_stopwords = stopwords.words("english")
symbols = punctuation + '№'
punctuation += '№'
punctuation += '®'

custom_top_words = ['мл','мг','шт','ooo','ру','лп','н','xch','bky','уп','%','п','изм','нд','p','р','n','\n','b','e','е','г','упаковка','пачка','серия','партия','рег','лср',
                    'pul',':','инвойс','rus','вэд','+','rg','g','ltd','d','c','д','кг','-','tyd','дата','оао','ао','зао','ип','инн','лс','код','окпд','россия','ул','область']
numbers = [i for i in range(10)]
numbers = list(map(str, numbers))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maximmezhov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
with open(config['vectorizer'], 'rb') as f:
    vectorizer = pickle.load(f)
    
with open(config['model'], 'rb') as f:
    model = pickle.load(f)

In [23]:
%%time
df = pd.DataFrame()
    
df['Наименование продукции'] = data[data.columns[1]]
df['precessed-text'] = data[data.columns[1]].apply(preprocess_text)

CPU times: user 15.6 s, sys: 344 ms, total: 15.9 s
Wall time: 37.8 s


In [24]:
%%time
X = vectorizer.transform(df['precessed-text'])
code = model.predict(X)
df['Код (определён автоматически)'] = (code/10).round(1)
df['Код (определён автоматически)'] = df['Код (определён автоматически)'].astype('str')
df['code'] = code
df.code = df.code.astype(int)
df['Подкатегория продукции'] = df['code'].map(code_dict)

CPU times: user 433 ms, sys: 10.9 ms, total: 444 ms
Wall time: 443 ms


In [25]:
f1_score(data[data.columns[2]].astype('str'), df['Код (определён автоматически)'], average='weighted')

0.9468498969389704

In [9]:
df

,Наименование продукции,precessed-text,Код (определён автоматически),code,Подкатегория продукции
0,смеси бетонные,смесь бетонный,2364.1,23641,Смеси строительные
1,ветеринарный препарат,ветеринарный препарат,9300.2,93002,Препараты и средства химико-фармацевтические д...
2,"Вибуркол, суппозитории ректальные гомеопатичес...",вибуркол суппозиторий ректальный гомеопатическ...,9300.1,93001,"Лекарственные средства, зарегистрированные в у..."
3,Неозим Prot-A 200 (Neozim Prot-А 200) – кормов...,неозим prot neozim prot – кормовой добавка пов...,9750.1,97501,Комбикорма и премиксы
4,"Вибуркол, суппозитории ректальные гомеопатичес...",вибуркол суппозиторий ректальный гомеопатическ...,9300.1,93001,"Лекарственные средства, зарегистрированные в у..."
...,...,...,...,...,...
59995,"Видеогастроскопы ""ПЕНТАКС"" ""EG"" c принадлежнос...",видеогастроскоп пентакса eg принадлежность,9442.3,94423,Приборы эндоскопические и увеличительные
59996,Рибоксин раствор для внутривенного введения 20...,рибоксин раствор внутривенный введение ампула ...,9300.1,93001,"Лекарственные средства, зарегистрированные в у..."
59997,"Анастрозол, таблетки покрытые пленочной оболоч...",анастрозол таблетка покрывать пленочный оболоч...,9300.1,93001,"Лекарственные средства, зарегистрированные в у..."
59998,"Стеклопакеты клееные строительного назначения,",стеклопакет клееный строительный назначение,5913.1,59131,Стеклопакеты


In [18]:
df[['Наименование продукции','Код (определён автоматически)','Подкатегория продукции']].to_csv('Результат анализа кодов по описанию продукции.csv')

# 1, 2 test
time check only

In [20]:
%%time
data = pd.read_excel('data/raw/2test.xlsx', engine='openpyxl')
df = pd.DataFrame() 
df['Наименование продукции'] = data[data.columns[1]]
df['precessed-text'] = data[data.columns[1]].apply(preprocess_text)
X = vectorizer.transform(df['precessed-text'])
code = model.predict(X)
df['Код (определён автоматически)'] = (code/10).round(1)
df['Код (определён автоматически)'] = df['Код (определён автоматически)'].astype('str')
df['code'] = code
df.code = df.code.astype(int)
df['Подкатегория продукции'] = df['code'].map(code_dict)

CPU times: user 1.29 s, sys: 28 ms, total: 1.32 s
Wall time: 3.92 s


In [21]:
f1_score(data[data.columns[2]].astype('str'), df['Код (определён автоматически)'], average='weighted')

0.8462352760699264

In [11]:
%%time
data = pd.read_excel('data/raw/1test.xlsx', engine='openpyxl')
df = pd.DataFrame() 
df['Наименование продукции'] = data[data.columns[1]]
df['precessed-text'] = data[data.columns[1]].apply(preprocess_text)
X = vectorizer.transform(df['precessed-text'])
code = model.predict(X)
df['Код (определён автоматически)'] = (code/10).round(1)
df['Код (определён автоматически)'] = df['Код (определён автоматически)'].astype('str')
df['code'] = code
df.code = df.code.astype(int)
df['Подкатегория продукции'] = df['code'].map(code_dict)

CPU times: user 39.6 ms, sys: 2.99 ms, total: 42.6 ms
Wall time: 61.6 ms


In [19]:
f1_score(data[data.columns[2]].astype('str'), df['Код (определён автоматически)'], average='weighted')

0.4666666666666667